# Data preparation

## Setup

In [1]:
import glob
import os
import tarfile

import pandas as pd
import yfinance as yf

In [2]:
# Get function that processes the datasets from yfinance
def process_yf_dataset(yf_ticker: str, title: str, value_col_name: str = "price", start_date: str = "2010-01-01", end_date: str = "2024-09-21") -> pd.DataFrame:
    # Fetch data from yfinance
    df = yf.download(yf_ticker, start=start_date, end=end_date)
    print(f"\nDataset for {yf_ticker} downloaded")

    # Export raw dataset
    raw_csv_path = f"~/Downloads/{title}_raw.csv"
    df.to_csv(os.path.expanduser(raw_csv_path), index=True)
    print(f"\nRaw dataset exported to '{raw_csv_path}'")

    # Data quality checks of relevant columns
    print("\nData quality checks:")
    print(f"- Consistent data types: {df[["Open", "High", "Low", "Close"]].dtypes.nunique() == 1}")
    print(f"- Null values:\n{df[["Open", "High", "Low", "Close"]].isna().sum()}")
    print(f"- Zero values:\n{(df[["Open", "High", "Low", "Close"]] == 0).sum()}")
    print(f"- Duplicated dates: {df[["Open", "High", "Low", "Close"]].index.duplicated().sum()}")

    # Get price using OHLC average ("adjusted close" is the same as "close" for the selected datasets)
    df[value_col_name] = df[["Open", "High", "Low", "Close"]].mean(axis=1)
    # Remove all the other columns, and rename index
    df = df.drop(columns=["Open", "High", "Low", "Close", "Adj Close", "Volume"]).rename_axis("date")
    print("\nDataset columns adjusted")

    # Export the final processed dataset
    final_csv_path = f"../data/{title}.csv"
    df.to_csv(final_csv_path, index=True)
    print(f"\nFinal dataset exported to '{final_csv_path}'")

    return df

In [3]:
def show_dataset_basic_info(df: pd.DataFrame) -> None:
    print("First and last entries of the dataset:")
    display(df.iloc[[0, -1]])

    print("\nBasic statistics of the dataset:")
    display(df.describe().T)

    median_days = df.index.year.value_counts().median()
    print(f"\nTypical number of trading days per year: {int(median_days)}")

## Bitcoin

**[Yahoo Finance](https://finance.yahoo.com/) provides BTC data only starting from 2014. Therefore, this [Coin Codex dataset](https://coincodex.com/crypto/bitcoin/historical-data) requires a different processing approach.**

In [4]:
# Match the path of the downloaded raw CSV
raw_btc_path = glob.glob(os.path.expanduser("~/Downloads/bitcoin_*.csv"))[0]

In [5]:
# Read raw CSV as df and show it
df_btc = pd.read_csv(raw_btc_path)
df_btc

,Start,End,Open,High,Low,Close,Volume,Market Cap
0,2024-09-20,2024-09-21,62935.8800,64053.9600,62400.0400,63102.0300,1.159340e+11,1.248770e+12
1,2024-09-19,2024-09-20,61831.9600,63819.9200,61592.0000,62953.9800,1.270354e+11,1.237745e+12
2,2024-09-18,2024-09-19,60313.9800,61449.1400,59216.0100,61449.1400,1.100440e+11,1.188172e+12
3,2024-09-17,2024-09-18,58267.9400,61259.9900,57620.0000,60303.5000,1.016688e+11,1.172633e+12
4,2024-09-16,2024-09-17,59191.9600,59191.9600,57549.7800,58229.2000,9.470302e+10,1.152253e+12
...,...,...,...,...,...,...,...,...
5175,2010-07-21,2010-07-22,0.0792,0.0792,0.0792,0.0792,0.000000e+00,2.743084e+05
5176,2010-07-20,2010-07-21,0.0747,0.0747,0.0747,0.0747,0.000000e+00,2.567102e+05
5177,2010-07-19,2010-07-20,0.0808,0.0808,0.0808,0.0808,0.000000e+00,2.775702e+05
5178,2010-07-18,2010-07-19,0.0858,0.0858,0.0858,0.0858,0.000000e+00,2.947466e+05


In [6]:
# Reverse the order of the rows
df_btc = df_btc[::-1].copy()

In [7]:
# Check the data type of each column
df_btc.dtypes

Start          object
End            object
Open          float64
High          float64
Low           float64
Close         float64
Volume        float64
Market Cap    float64
dtype: object

In [8]:
# Check if the data type of each relevant column is consistent across all rows
df_btc[["Start", "Open", "High", "Low", "Close"]].map(type).nunique() == 1

Start    True
Open     True
High     True
Low      True
Close    True
dtype: bool

In [9]:
# Check for null values in each relevant column
df_btc[["Start", "Open", "High", "Low", "Close"]].isna().sum()

Start    0
Open     0
High     0
Low      0
Close    0
dtype: int64

In [10]:
# Check for zero values in each relevant column
(df_btc[["Start", "Open", "High", "Low", "Close"]] == 0).sum()

Start    0
Open     0
High     0
Low      0
Close    0
dtype: int64

In [11]:
# Get price using OHLC average
df_btc["price"] = df_btc[["Open", "High", "Low", "Close"]].mean(axis=1)

In [12]:
# Rename date column and remove all the other columns
df_btc.rename(columns={"Start": "date"}, inplace=True)
df_btc.drop(columns=["End", "Open", "High", "Low", "Close", "Volume", "Market Cap"], inplace=True)

In [13]:
# Check if dates progress correctly assuming all days of the year (without skipped or duplicated dates)
df_btc["date"] = pd.to_datetime(df_btc["date"])
date_diff = df_btc["date"].diff().dropna()
(date_diff == pd.Timedelta(days=1)).all()

True

In [14]:
# Make date as index and show final df
df_btc.set_index("date", inplace=True)
df_btc

,price
date,
2010-07-17,0.0500
2010-07-18,0.0858
2010-07-19,0.0808
2010-07-20,0.0747
2010-07-21,0.0792
...,...
2024-09-16,58540.7250
2024-09-17,59362.8575
2024-09-18,60607.0675


In [15]:
# Export final df
df_btc.to_csv("../data/BTC.csv", index=True)

In [16]:
show_dataset_basic_info(df_btc)

First and last entries of the dataset:


,price
date,
2010-07-17,0.0500
2024-09-20,63122.9775



Basic statistics of the dataset:


,count,mean,std,min,25%,50%,75%,max
price,5180.0,12632.49931,18358.368932,0.05,233.617188,3427.284242,19225.075768,72372.82



Typical number of trading days per year: 365


## S&P 500

In [17]:
df_sp500 = process_yf_dataset("^GSPC", "SP500")

[*********************100%***********************]  1 of 1 completed


Dataset for ^GSPC downloaded

Raw dataset exported to '~/Downloads/SP500_raw.csv'

Data quality checks:
- Consistent data types: True
- Null values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Zero values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Duplicated dates: 0

Dataset columns adjusted

Final dataset exported to '../data/SP500.csv'


In [18]:
show_dataset_basic_info(df_sp500)

First and last entries of the dataset:


,price
date,
2010-01-04,1124.995026
2024-09-20,5700.455078



Basic statistics of the dataset:


,count,mean,std,min,25%,50%,75%,max
price,3704.0,2667.808033,1198.662166,1024.777496,1689.163124,2390.29248,3697.753174,5709.064941



Typical number of trading days per year: 252


## US 10-year treasury yield

In [19]:
df_us10y = process_yf_dataset("^TNX", "US10Y", "yield")

[*********************100%***********************]  1 of 1 completed


Dataset for ^TNX downloaded

Raw dataset exported to '~/Downloads/US10Y_raw.csv'

Data quality checks:
- Consistent data types: True
- Null values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Zero values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Duplicated dates: 0

Dataset columns adjusted

Final dataset exported to '../data/US10Y.csv'


In [20]:
show_dataset_basic_info(df_us10y)

First and last entries of the dataset:


,yield
date,
2010-01-04,3.84175
2024-09-20,3.73525



Basic statistics of the dataset:


,count,mean,std,min,25%,50%,75%,max
yield,3702.0,2.459896,0.911333,0.4955,1.813562,2.3575,2.96675,4.9605



Typical number of trading days per year: 251


## Gold (futures)

In [21]:
df_au = process_yf_dataset("GC=F", "AU")

[*********************100%***********************]  1 of 1 completed


Dataset for GC=F downloaded

Raw dataset exported to '~/Downloads/AU_raw.csv'

Data quality checks:
- Consistent data types: True
- Null values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Zero values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Duplicated dates: 0

Dataset columns adjusted

Final dataset exported to '../data/AU.csv'


In [22]:
show_dataset_basic_info(df_au)

First and last entries of the dataset:


,price
date,
2010-01-04,1113.699982
2024-09-20,2605.624939



Basic statistics of the dataset:


,count,mean,std,min,25%,50%,75%,max
price,3701.0,1523.826864,318.853363,1054.524963,1257.025024,1416.925018,1776.399994,2605.624939



Typical number of trading days per year: 252


## Crude oil (futures)

In [23]:
df_wti = process_yf_dataset("CL=F", "WTI")

[*********************100%***********************]  1 of 1 completed


Dataset for CL=F downloaded

Raw dataset exported to '~/Downloads/WTI_raw.csv'

Data quality checks:
- Consistent data types: True
- Null values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Zero values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Duplicated dates: 0

Dataset columns adjusted

Final dataset exported to '../data/WTI.csv'


In [24]:
show_dataset_basic_info(df_wti)

First and last entries of the dataset:


,price
date,
2010-01-04,80.612499
2024-09-20,71.945000



Basic statistics of the dataset:


,count,mean,std,min,25%,50%,75%,max
price,3702.0,72.026457,21.507457,-10.5925,53.05625,73.56625,89.982499,122.719999



Typical number of trading days per year: 252


## USD/CHF

In [25]:
df_chf = process_yf_dataset("CHF=X", "CHF", "rate")

[*********************100%***********************]  1 of 1 completed


Dataset for CHF=X downloaded

Raw dataset exported to '~/Downloads/CHF_raw.csv'

Data quality checks:
- Consistent data types: True
- Null values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Zero values:
Open     0
High     0
Low      0
Close    0
dtype: int64
- Duplicated dates: 0

Dataset columns adjusted

Final dataset exported to '../data/CHF.csv'


In [26]:
show_dataset_basic_info(df_chf)

First and last entries of the dataset:


,rate
date,
2010-01-01,1.035025
2024-09-20,0.848102



Basic statistics of the dataset:


,count,mean,std,min,25%,50%,75%,max
rate,3833.0,0.948016,0.051927,0.72367,0.912402,0.947785,0.983642,1.162888



Typical number of trading days per year: 260


## Archive the raw data

In [27]:
# List of filenames for raw CSV data
csv_titles = ["SP500", "US10Y", "AU", "WTI", "CHF"]

# Generate full paths of the raw CSV files in the downloads dir
csv_file_paths = [os.path.join(os.path.expanduser("~/Downloads/"), f"{csv_title}_raw.csv") for csv_title in csv_titles]

# Append the bitcoin raw CSV path to the list
csv_file_paths.append(raw_btc_path)

# Create a compressed tar archive of the raw CSV files
with tarfile.open("../data/.raw_data.tar.xz", "w:xz") as archive:
    for csv_file_path in csv_file_paths:
        archive.add(csv_file_path, arcname=os.path.basename(csv_file_path))